# Parsing APIs Example

## Intro

Now we will take a look on a real data. When you parse data from web you will often meet API based web-pages. 

For example [zalando.fr](https://www.zalando.fr/accueil-homme/) is API based web-page. 

In this guided lab you will learn how to obtain the links from webpages and extract the data. Read through this doc, execute the cells in order and make sure you understand the explanations. 

*Note: This guided lab uses Google Chrome. Other browsers like Safari and Firefox have similar tools for developers but they work differently. To save your time in following this lab, it is strongly recommended that you install and use Google Chrome.*

## Obtaining the link

Zalando is discount e-store where you can buy clothes and accesories with discount. When we go to the web-page, we can choose different sections. First the general process will be shown using [Children section](https://www.zalando.fr/accueil-enfant/) as example.

Here we will parse data about promotions only. Therefore, final output will be the DataFrame with all the goods under discount.

[![Image from Gyazo](https://i.gyazo.com/fa4874d8e81c7570273bbfb853d66308.png)](https://gyazo.com/fa4874d8e81c7570273bbfb853d66308)


We go to Promos page. Right click of mouse shows us a list of actions possible, from which we select Inspect.

<img src='https://i.gyazo.com/bccbd11d69c9040dc98758d443e32052.png' width="400">


You will see the menu dropdown on the right side or on the bottom of the window. There you should click on Network:


[![Image from Gyazo](https://i.gyazo.com/f7e0db81cbfee67694183d1a7640bf81.png)](https://gyazo.com/f7e0db81cbfee67694183d1a7640bf81)

Right after the developer part will change showing the files behind the page. In order to obtain only useful files we select the following settings:
1. Preserve Log
2. Select XHR files.

[![Image from Gyazo](https://i.gyazo.com/9a899d4441d9d93e795f79747f1e47d5.png)](https://gyazo.com/9a899d4441d9d93e795f79747f1e47d5)

In order to obtain some files we need to scrool down and go forward to second page. 

[![Image from Gyazo](https://i.gyazo.com/0956eb3d5125075a236c9a439c7749c7.png)](https://gyazo.com/0956eb3d5125075a236c9a439c7749c7)

In the Network panel you can see the following files being uploaded. All the data on the web-page is uploaded from the json file, which is one of the following. It is important to understand which file contains what kind of information. 

<a href="https://gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c"><img src="https://i.gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c.png" alt="Image from Gyazo" width="724.8"/></a>

When you find what kind of information you need for the data to be uploaded you just test it. Here we need the article... file:

<a href="https://gyazo.com/78b35bf492994b3f35c0564a21da202a"><img src="https://i.gyazo.com/78b35bf492994b3f35c0564a21da202a.png" alt="Image from Gyazo" width="727.2"/></a>

When we test the link in Chrome inkognito mode we obtain the proper json file:


<a href="https://gyazo.com/b60453fa98454fa29771c731a5174443"><img src="https://i.gyazo.com/b60453fa98454fa29771c731a5174443.png" alt="Image from Gyazo" width="1530.4"/></a>

In order to change the objects in the json file (kind of pagination), you need to change the offset (the number of the first element on the page). in fact, if you take a look on the link, it is easy to unerstand the structure of the link.

# Reading the data

Now the party rocks! When we know how can we obtain the data, it is not a problem to obtain the whole database with all the data from the web-page.
In this lab you will collect your database of Zalando products. You select which goods you want to track. You can define as many filters to your data as you want. Just make sure that the data represents the filters.




In [5]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import urllib.request

In [6]:
# Paste the url you obtained for your data
url = 'https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=1&sort=popularity'


#### Collect first 84 object of the of the data (1st page)

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having *family_articles, flags, media* and *sizes* remaining lists (they are exceptions). Hint: use the headers parameter to get the data!

In [7]:
# headers definition

headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}

In [11]:
req = requests.get(url, headers=headers)

enfants = urllib.request.urlopen(url)
mybytes = enfants.read()

response = mybytes.decode('utf8')
enfants.close()

In [20]:
# Your code
zalando = json.loads(response)
zalando

{'total_count': 25935,
 'pagination': {'page_count': 309, 'current_page': 1, 'per_page': 84},
 'sort': 'popularity',
 'articles': [{'sku': 'LE226G007-A11',
   'name': 'BATWING - T-shirt à manches longues - white',
   'price': {'original': '21,95\xa0€',
    'promotional': '15,35\xa0€',
    'has_different_prices': True,
    'has_different_original_prices': False,
    'has_different_promotional_prices': True,
    'has_discount_on_selected_sizes_only': False},
   'sizes': ['3a', '5a', '6a', '8a', '10a', '12a', '14a', '16a'],
   'url_key': 'levisr-batwing-tee-t-shirt-a-manches-longues-le226g007-a11',
   'media': [{'path': 'LE/22/6G/00/7A/11/LE226G007-A11@2.jpg',
     'role': 'DEFAULT',
     'packet_shot': False}],
   'brand_name': "Levi's®",
   'is_premium': False,
   'family_articles': [],
   'flags': [{'key': 'discountRate',
     'value': '-30%',
     'tracking_value': 'discount rate'}],
   'product_group': 'clothing',
   'delivery_promises': []},
  {'sku': 'N1243A0ZC-Q11',
   'name': "PI

In [13]:
data = pd.json_normalize(zalando)
data

,total_count,sort,articles,query_path,next_page_path,page_gender,premium,filters,total_article_count,plusStatus,...,inCatalogTeaser.content.tracking_data.cf_tracking_params.time_unit,inCatalogTeaser.content.tracking_data.cf_tracking_params.time_unit_count,inCatalogTeaser.content.tracking_data.cf_tracking_params.shop_slot_id,inCatalogTeaser.content.tracking_data.cf_tracking_params.campaign_source_id,inCatalogTeaser.content.tracking_data.cf_tracking_params.request_id,inCatalogTeaser.content.tracking_data.cf_tracking_params.campaign_id,inCatalogTeaser.content.tracking_data.cf_tracking_params.creative_id,inCatalogTeaser.content.click_url,inCatalogTeaser.channel_name,inCatalogTeaser.slot_id
0,25935,popularity,"[{'sku': 'LE226G007-A11', 'name': 'BATWING - T...",/promo-enfant/,/promo-enfant/?p=2,kids,False,"[{'key': 'sizes', 'label': 'Taille', 'url_key'...",25936,non-eligible,...,DAY,1,in_catalog,undefined,bb353816-b9f9-480d-b198-01e28483e84a,85ee0202-3a51-463b-a231-7352e27efe67,88f22699-8309-496f-a9f9-07c1b546bc59,https://www.zalando.fr/collections/yPgnX9M1TQi...,CF,cb224cac-736d-4e00-b757-d8a8007ef951


In [22]:
#### Collect all the objects from selected filters. Total number of pages can be found in the same json. Use *sku* column as index.Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having family_articles, flags, media and sizes remaining lists (they are exceptions).

dataframe=pd.json_normalize(zalando['articles'])
dataframe

,sku,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,delivery_promises,price.original,price.promotional,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,amount
0,LE226G007-A11,BATWING - T-shirt à manches longues - white,"[3a, 5a, 6a, 8a, 10a, 12a, 14a, 16a]",levisr-batwing-tee-t-shirt-a-manches-longues-l...,[{'path': 'LE/22/6G/00/7A/11/LE226G007-A11@2.j...,Levi's®,False,[],"[{'key': 'discountRate', 'value': '-30%', 'tra...",clothing,[],"21,95 €","15,35 €",True,False,True,False,NaN
1,N1243A0ZC-Q11,PICO 5 - Chaussures d'entraînement et de fitne...,"[27.5, 28.5, 29.5, 30, 31.5, 32, 33, 33.5, 34,...",nike-performance-pico-5-chaussures-dentraineme...,[{'path': 'N1/24/3A/0Z/CQ/11/N1243A0ZC-Q11@11....,Nike Performance,False,[],"[{'key': 'discountRate', 'value': '-15%', 'tra...",shoe,[],"29,95 €","25,45 €",False,False,False,False,NaN
2,NI113D09V-Q12,AIR MAX 270 EXTREME - Mocassins - black,"[27.5, 28, 28.5, 29.5, 30]",nike-sportswear-air-max-270-extreme-baskets-ba...,[{'path': 'NI/11/3D/09/VQ/12/NI113D09V-Q12@16....,Nike Sportswear,False,[],"[{'key': 'discountRate', 'value': '-50%', 'tra...",shoe,[],"89,95 €","44,95 €",False,False,False,False,NaN
3,NI114D0C1-Q12,AIR MAX TAILWIND IV - Baskets basses - black,"[36.5, 38, 38.5, 39, 40]",nike-sportswear-air-max-tailwind-baskets-basse...,[{'path': 'NI/11/4D/0C/1Q/12/NI114D0C1-Q12@10....,Nike Sportswear,False,[],"[{'key': 'discountRate', 'value': '-20%', 'tra...",shoe,[],"139,95 €","111,96 €",False,False,False,False,NaN
4,NI114D0A2-Q11,SHOX R4 - Baskets basses - black,"[36.5, 37.5, 38, 38.5, 39, 40]",nike-sportswear-nike-shox-r4-baskets-basses-ni...,[{'path': 'NI/11/4D/0A/2Q/11/NI114D0A2-Q11@7.j...,Nike Sportswear,False,[],"[{'key': 'discountRate', 'value': '-55%', 'tra...",shoe,[],"119,95 €","53,95 €",False,False,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,NI114D0AW-A12,AIR MAX 270 RT - Baskets basses - white/pink/l...,"[19.5, 21, 22, 23.5, 25, 26]",nike-sportswear-air-max-270-react-baskets-bass...,[{'path': 'NI/11/4D/0A/WA/12/NI114D0AW-A12@6.j...,Nike Sportswear,False,[],"[{'key': 'discountRate', 'value': '-30%', 'tra...",shoe,[],"74,95 €","52,45 €",False,False,False,False,NaN
80,4BE24A062-K12,Jean slim - blue denim,"[3-4a, 6-7a, 7-8a, 8-9a, 11-12a]",benetton-jean-slim-blue-denim-4be24a062-k12,[{'path': '4B/E2/4A/06/2K/12/4BE24A062-K12@5.j...,Benetton,False,[],"[{'key': 'discountRate', 'value': '-35%', 'tra...",clothing,[],"25,95 €","16,95 €",False,False,False,False,NaN
81,NI114D0A3-A11,AIR MAX AXIS - Baskets basses - white/total or...,"[35.5, 37.5, 38, 38.5, 39, 40]",nike-sportswear-nike-air-max-axis-baskets-bass...,[{'path': 'NI/11/4D/0A/3A/11/NI114D0A3-A11@12....,Nike Sportswear,False,[],"[{'key': 'discountRate', 'value': '-35%', 'tra...",shoe,[],"84,95 €","54,95 €",False,False,False,False,NaN
82,LE226G00Z-A11,T-shirt imprimé - white,"[3a, 5a, 6a, 8a, 10a, 12a, 14a, 16a]",levis-t-shirt-imprime-white-le226g00z-a11,[{'path': 'LE/22/6G/00/ZA/11/LE226G00Z-A11@5.j...,Levi's®,False,[],"[{'key': 'discountRate', 'value': '-35%', 'tra...",clothing,[],"22,95 €","14,95 €",True,False,True,False,NaN


In [23]:
# Get the total number of pages

total_pages = zalando['pagination']['page_count']
total_pages

309

#### Display the trending brand in DataFrame

In [24]:
dataframe["brand_name"].value_counts()

Nike Sportswear          18
Levi's®                   9
Nike Performance          7
BOSS Kidswear             4
Champion                  4
adidas Performance        4
Friboo                    3
Lacoste                   3
Fila                      3
Jack & Jones Junior       3
adidas Originals          3
Timberland                2
Kaporal                   2
Benetton                  2
Catimini                  2
Re-Gen                    2
Converse                  2
Emporio Armani            1
Quiksilver                1
Tommy Hilfiger            1
Calvin Klein Swimwear     1
Lacoste Sport             1
Blue Seven                1
Skechers                  1
Jordan                    1
D-XEL                     1
Vans                      1
Esprit                    1
Name: brand_name, dtype: int64

#### Display the brand with maximal total discount (sum of discounts on all goods)

In [ ]:
#Our data is still text. Convert prices into numbers.
# your code

#### Display the brands without discount at all

In [ ]:
# your code